# **Importing Libraries & Reading Data**

In [ ]:
import pandas as pd
import seaborn as sb
import numpy as np
from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
a1=pd.read_csv("../input/spaceship-titanic/train.csv")

In [ ]:
a1.head(3)

In [ ]:
a1.shape

In [ ]:
a1.nunique()

In [ ]:
a1.info()

In [ ]:
a1["CryoSleep"]=a1["CryoSleep"].astype(str)

# **Missing Data Treatment**

In [ ]:
a1.isna().sum()

In [ ]:
a1["Name"]=a1["Name"].fillna("Undisclosed")

In [ ]:
def replacer(df):
  for i in df.columns:
    if df[i].dtypes=="object":
      x=list(df[i].mode())[0]
      df[i]=df[i].fillna(x)
    else:
      x=df[i].mean()
      df[i]=df[i].fillna(x)
  return df.isna().sum()

In [ ]:
replacer(a1)

# **Outlier Analysis**

In [ ]:
def outlier_analysis(df):
  cat=[]
  con=[]
  for i in df.columns:
    if df[i].dtypes=="object":
      cat.append(i)
    else:
      con.append(i)
  
  import pandas as pd
  from sklearn.preprocessing import StandardScaler
  ss=StandardScaler()
  A1=pd.DataFrame(ss.fit_transform(df[con]),columns=con)
  A2=pd.DataFrame(df[cat],columns=cat)

  OUT=pd.DataFrame(A1[(A1>3)].sum().sort_values(),columns=["OUT_VAL"])
  return OUT

In [ ]:
outlier_analysis(a1)

### **We need to eliminate Outliers from;**

Age	96.327240

ShoppingMall	725.946591

RoomService	912.430461

FoodCourt	947.161216

VRDeck	970.926634

Spa	980.108266

VIP	1300.117687

# **Eliminating Outliers**

In [ ]:
def outliers_rem(df):
  cat=[]
  con=[]
  for i in df.columns:
    if df[i].dtypes=="object":
      cat.append(i)
    else:
      con.append(i)
  
  import pandas as pd
  from sklearn.preprocessing import StandardScaler
  ss=StandardScaler()
  A1=pd.DataFrame(ss.fit_transform(df[con]),columns=con)
  A2=pd.DataFrame(df[cat],columns=cat)

  OUT=pd.DataFrame(A1[(A1>3)].sum().sort_values(),columns=["OUT_VAL"])
  OUT_Fea=list(OUT[OUT["OUT_VAL"]>=2.9].index)

  A=A1.join(A2)

  for i in OUT_Fea:
    A=A[(A[i]<3)&(A[i]>-3)]
  
  A3=pd.DataFrame(ss.inverse_transform(A[con]),columns=con)
  A4=pd.DataFrame(A[cat],columns=cat)
  a=A3.join(A4)
  print("___________________________________________")
  print("Features with Outliers: ",OUT_Fea)
  print("___________________________________________")
  return a

In [ ]:
a=outliers_rem(a1)

In [ ]:
a.shape

In [ ]:
a.index=range(0,7798)

In [ ]:
a1.shape

# **Univarite Analysis**

In [ ]:
U=a.drop(labels=["PassengerId","Cabin","Name"],axis=1)

In [ ]:
def UNI(df,fs):
  def catcon(df):
    cat=[]
    con=[]
    for i in df.columns:
      if df[i].dtypes=="object":
        cat.append(i)
      else:
        con.append(i)
    return cat,con
  cat,con=catcon(df)

  import pandas as pd
  import seaborn as sb
  import matplotlib.pyplot as plt
  plt.figure(figsize=fs)
  cols=3
  tot_rows=df.shape[1]
  rows=tot_rows//cols

  plt_no=1
  for i in con:
    plt.subplot(rows,cols,plt_no)
    sb.distplot(df[i])
    plt_no+=1
  plt_no=1
  for i in cat:
    plt.subplot(rows,cols,plt_no)
    q=sb.countplot(df[i])
    q.set_xticklabels(q.get_xticklabels(),rotation=21,horizontalalignment="right")
    plt_no+=1

In [ ]:
UNI(U,(20,20))

# **Bivariate Analysis w.r.t. Transported**

In [ ]:
def BIV_H(df,fs,con_col):
  def catcon(df):
    cat=[]
    con=[]
    for i in df.columns:
      if df[i].dtypes=="object":
        cat.append(i)
      else:
        con.append(i)
    return cat,con
  cat,con=catcon(df)

  import pandas as pd
  import seaborn as sb
  import matplotlib.pyplot as plt
  plt.figure(figsize=fs)
  cols=3
  tot_rows=df.shape[1]
  rows=tot_rows//cols

  plt_no=1
  for i in con:
    plt.subplot(rows,cols,plt_no)
    sb.scatterplot(df[i],df[con_col])
    plt_no+=1
  plt_no=1
  for i in cat:
    plt.subplot(rows,cols,plt_no)
    q=sb.boxplot(df[i],df[con_col])
    q.set_xticklabels(q.get_xticklabels(),rotation=21,horizontalalignment="right")
    plt_no+=1

In [ ]:
BIV_H(U,(20,15),"Transported")

# **Heatmap**

In [ ]:
def HM(df,fs):
  import seaborn as sb
  import matplotlib.pyplot as plt
  plt.figure(figsize=fs)
  sb.heatmap(df.corr(),annot=True,cmap="coolwarm")

In [ ]:
HM(a,(20,10))

# **Continuous and Categorical columns**

In [ ]:
def catcon(df):
  cat=[]
  con=[]
  for i in df.columns:
    if df[i].dtypes=="object":
      cat.append(i)
    else:
      con.append(i)
  return cat,con

In [ ]:
cat,con=catcon(a)

In [ ]:
cat

In [ ]:
con

# **ChiSquare Test**

In [ ]:
def CHI(df,cat_col):
  from scipy.stats import chi2_contingency
  import pandas as pd
  def catcon(df):
    cat=[]
    con=[]
    for i in df.columns:
      if df[i].dtypes=="object":
        cat.append(i)
      else:
        con.append(i)
    return cat,con
  cat,con=catcon(df)
  imp_cat=[]
  for i in cat:
    q=chi2_contingency(pd.crosstab(df[cat_col],df[i]))[1]
    if q < 0.035:
      imp_cat.append(i)
  for i in imp_cat:
    if cat_col in imp_cat:
      imp_cat.remove(cat_col)
    else:
      pass
  return imp_cat

In [ ]:
CHI(a,"Transported")

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(a["Transported"],a["PassengerId"]))[1]

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(a["Transported"],a["HomePlanet"]))[1]

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(a["Transported"],a["CryoSleep"]))[1]

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(a["Transported"],a["Cabin"]))[1]

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(a["Transported"],a["Destination"]))[1]

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(a["Transported"],a["VIP"]))[1]

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(a["Transported"],a["Name"]))[1]

# **Anova Test**

In [ ]:
def ANOVA_cat(df,cat_col):
  from statsmodels.formula.api import ols
  from statsmodels.stats.anova import anova_lm
  def catcon(df):
    cat=[]
    con=[]
    for i in df.columns:
      if df[i].dtypes=="object":
        cat.append(i)
      else:
        con.append(i)
    return cat,con
  cat,con=catcon(df)

  imp_con=[]
  for i in con:
    reg=i + "~" + cat_col
    model=ols(reg,df).fit()
    q=anova_lm(model)
    if q.iloc[0,4] < 0.04:
      imp_con.append(i)
  for i in imp_con:
    if cat_col in imp_con:
      imp_con.remove(cat_col)
    else:
      pass
  return imp_con

In [ ]:
imp_con=ANOVA_cat(a,"Transported")

In [ ]:
imp_con

In [ ]:
imp_cat=["Destination","CryoSleep"]

# **Defining X and Y**

In [ ]:
X=a[imp_con]

In [ ]:
Y=a[["Transported"]]

# **Preprocessing**

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

In [ ]:
X1=pd.DataFrame(ss.fit_transform(X[imp_con]),columns=imp_con)

In [ ]:
Xnew=X1

In [ ]:
Xnew.head(3)

# **Train Test Split**

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

# **Decision Tree MSplit**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(criterion="gini",random_state=31,min_samples_split=11)
model_dtc=dtc.fit(xtrain,ytrain)

pred_tr_dtc = model_dtc.predict(xtrain)
pred_ts_dtc = model_dtc.predict(xtest)

from sklearn.metrics import accuracy_score
tr_acc_dtc = accuracy_score(ytrain,pred_tr_dtc)
ts_acc_dtc = accuracy_score(ytest,pred_ts_dtc)

print("__________________________________________")
print("Training Accuracy: ",tr_acc_dtc)
print("Testing Accuracy: ",ts_acc_dtc)
print("__________________________________________")

### **Finding Best Parameters for DTC**

In [ ]:
tg={"criterion":["entropy","gini"],"min_samples_split":range(0,60)}

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(random_state=31)
from sklearn.model_selection import GridSearchCV
cv=GridSearchCV(dtc,tg,scoring="accuracy",cv=4)
cvmodel=cv.fit(xtrain,ytrain)
cvmodel.best_params_

**Msplit Model with best params**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(criterion="gini",random_state=31,min_samples_split=50)
model_dtc_ms=dtc.fit(xtrain,ytrain)

pred_tr_dtc_ms = model_dtc_ms.predict(xtrain)
pred_ts_dtc_ms = model_dtc_ms.predict(xtest)

from sklearn.metrics import accuracy_score
tr_acc_dtc_ms = accuracy_score(ytrain,pred_tr_dtc_ms)
ts_acc_dtc_ms = accuracy_score(ytest,pred_ts_dtc_ms)

print("__________________________________________")
print("Training Accuracy: ",tr_acc_dtc_ms)
print("Testing Accuracy: ",ts_acc_dtc_ms)
print("__________________________________________")

# **Decision Tree Mleaf**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(criterion="gini",random_state=31,min_samples_leaf=15)
model_dtc=dtc.fit(xtrain,ytrain)

pred_tr_dtc = model_dtc.predict(xtrain)
pred_ts_dtc = model_dtc.predict(xtest)

from sklearn.metrics import accuracy_score
tr_acc_dtc = accuracy_score(ytrain,pred_tr_dtc)
ts_acc_dtc = accuracy_score(ytest,pred_ts_dtc)

print("__________________________________________")
print("Training Accuracy: ",tr_acc_dtc)
print("Testing Accuracy: ",ts_acc_dtc)
print("__________________________________________")

In [ ]:
tg={"criterion":["entropy","gini"],"min_samples_leaf":range(0,60)}

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(random_state=31)
from sklearn.model_selection import GridSearchCV
cv=GridSearchCV(dtc,tg,scoring="accuracy",cv=4)
cvmodel=cv.fit(xtrain,ytrain)
cvmodel.best_params_

**Mleaf Model with best params**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(criterion="entropy",random_state=31,min_samples_leaf=33)
model_dtc_mleaf=dtc.fit(xtrain,ytrain)

pred_tr_dtc_mleaf = model_dtc_ms.predict(xtrain)
pred_ts_dtc_mleaf = model_dtc_ms.predict(xtest)

from sklearn.metrics import accuracy_score
tr_acc_dtc_mleaf = accuracy_score(ytrain,pred_tr_dtc_mleaf)
ts_acc_dtc_mleaf = accuracy_score(ytest,pred_ts_dtc_mleaf)

print("__________________________________________")
print("Training Accuracy: ",tr_acc_dtc_mleaf)
print("Testing Accuracy: ",ts_acc_dtc_mleaf)
print("__________________________________________")

# **Decision Tree MaxDepth**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(criterion="gini",random_state=31,max_depth=3)
model_dtc=dtc.fit(xtrain,ytrain)

pred_tr_dtc = model_dtc.predict(xtrain)
pred_ts_dtc = model_dtc.predict(xtest)

from sklearn.metrics import accuracy_score
tr_acc_dtc = accuracy_score(ytrain,pred_tr_dtc)
ts_acc_dtc = accuracy_score(ytest,pred_ts_dtc)

print("__________________________________________")
print("Training Accuracy: ",tr_acc_dtc)
print("Testing Accuracy: ",ts_acc_dtc)
print("__________________________________________")

In [ ]:
tg={"criterion":["entropy","gini"],"max_depth":range(0,20)}

from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(random_state=31)
from sklearn.model_selection import GridSearchCV
cv=GridSearchCV(dtc,tg,scoring="accuracy",cv=4)
cvmodel=cv.fit(xtrain,ytrain)
cvmodel.best_params_

**Mdepth Model with best params**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(criterion="entropy",random_state=31,max_depth=8)
model_dtc_mdepth=dtc.fit(xtrain,ytrain)

pred_tr_dtc_mdepth = model_dtc_mdepth.predict(xtrain)
pred_ts_dtc_mdepth = model_dtc_mdepth.predict(xtest)

from sklearn.metrics import accuracy_score
tr_acc_dtc_mdepth = accuracy_score(ytrain,pred_tr_dtc_mdepth)
ts_acc_dtc_mdepth = accuracy_score(ytest,pred_ts_dtc_mdepth)

print("__________________________________________")
print("Training Accuracy: ",tr_acc_dtc_mdepth)
print("Testing Accuracy: ",ts_acc_dtc_mdepth)
print("__________________________________________")

# **Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(criterion="gini",random_state=31,min_samples_split=11,n_estimators=50)
model_rfc=rfc.fit(xtrain,ytrain)

pred_tr_rfc=model_rfc.predict(xtrain)
pred_ts_rfc=model_rfc.predict(xtest)

from sklearn.metrics import accuracy_score
tr_acc_rfc = accuracy_score(ytrain,pred_tr_rfc)
ts_acc_rfc = accuracy_score(ytest,pred_ts_rfc)

print("__________________________________________")
print("Training Accuracy: ",tr_acc_rfc)
print("Testing Accuracy: ",ts_acc_rfc)
print("__________________________________________")

In [ ]:
tg={"min_samples_split":range(2,20),"n_estimators":range(2,20)}

from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=31)
from sklearn.model_selection import GridSearchCV
cv=GridSearchCV(rfc,tg,scoring="accuracy",cv=4)
cvmodel=cv.fit(xtrain,ytrain)
cvmodel.best_params_

**RFC Model with best params**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(criterion="gini",random_state=31,min_samples_split=15,n_estimators=19)
model_rfc=rfc.fit(xtrain,ytrain)

pred_tr_rfc=model_rfc.predict(xtrain)
pred_ts_rfc=model_rfc.predict(xtest)

from sklearn.metrics import accuracy_score
tr_acc_rfc = accuracy_score(ytrain,pred_tr_rfc)
ts_acc_rfc = accuracy_score(ytest,pred_ts_rfc)

print("__________________________________________")
print("Training Accuracy: ",tr_acc_rfc)
print("Testing Accuracy: ",ts_acc_rfc)
print("__________________________________________")

# **AdaBoost**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

dtc=DecisionTreeClassifier(criterion='gini',min_samples_split=57,random_state=31)
abc=AdaBoostClassifier(dtc,n_estimators=50)
model_abc=abc.fit(xtrain,ytrain)

pred_tr_abc=model_abc.predict(xtrain)
pred_ts_abc=model_abc.predict(xtest)

from sklearn.metrics import accuracy_score
tr_acc_abc = accuracy_score(ytrain,pred_tr_abc)
ts_acc_abc = accuracy_score(ytest,pred_ts_abc)

print("__________________________________________")
print("Training Accuracy: ",tr_acc_abc)
print("Testing Accuracy: ",ts_acc_abc)
print("__________________________________________")

In [ ]:
tg={"n_estimators":range(2,60)}

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

dtc=DecisionTreeClassifier(criterion='gini',min_samples_split=57,random_state=31)
abc=AdaBoostClassifier(dtc)

from sklearn.model_selection import GridSearchCV
cv=GridSearchCV(abc,tg,scoring="accuracy",cv=4)
cvmodel=cv.fit(xtrain,ytrain)
cvmodel.best_params_

**AdaBoost Model with best params**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

dtc=DecisionTreeClassifier(criterion='gini',min_samples_split=57,random_state=31)
abc=AdaBoostClassifier(dtc,n_estimators=2)
model_abc=abc.fit(xtrain,ytrain)

pred_tr_abc=model_abc.predict(xtrain)
pred_ts_abc=model_abc.predict(xtest)

from sklearn.metrics import accuracy_score
tr_acc_abc = accuracy_score(ytrain,pred_tr_abc)
ts_acc_abc = accuracy_score(ytest,pred_ts_abc)

print("__________________________________________")
print("Training Accuracy: ",tr_acc_abc)
print("Testing Accuracy: ",ts_acc_abc)
print("__________________________________________")

# **Conclusion:**

## **The best models are Decision Tree Classifier with max_depth value = 8** 
---
## **And Random Forest Classifier with params:**
## **min_samples_split = 15, n_estimators = 19**
---
## **Now we use the model to predict on testing data**

# **Preparing Testing Data**

In [ ]:
B1=pd.read_csv("../input/spaceship-titanic/test.csv")

In [ ]:
B1["CryoSleep"]=B1["CryoSleep"].astype(str)

In [ ]:
B=B1[imp_con]

In [ ]:
B.head(3)

# **Missing Data Treatment**

In [ ]:
replacer(B)

# **Preprocessing**

### **Note: Use only transform method**

In [ ]:
X1=pd.DataFrame(ss.fit_transform(X[imp_con]),columns=imp_con)

In [ ]:
B2=pd.DataFrame(ss.transform(B[imp_con]),columns=imp_con)

Bnew=B2

In [ ]:
Bnew.head()

In [ ]:
Xnew.shape

In [ ]:
Bnew.shape

In [ ]:
Bfinal=Bnew[Xnew.columns]

# **Predicting with RFC**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(criterion="gini",random_state=31,min_samples_split=15,n_estimators=19)
model_rfc=rfc.fit(Xnew,Y)

predicted_rfc=model_rfc.predict(Bfinal)

In [ ]:
T2=B1[["PassengerId"]]


In [ ]:
T2["Transported"]=predicted_rfc

In [ ]:
T2["Transported"].replace([1.0,0.0],["True","False"],inplace=True)

In [ ]:
T2.set_index("PassengerId",inplace=True)

In [ ]:
T2

# **-----------------END----------------------------**